In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pprint

In [2]:
"""
TODOs:
    solve for displacements of a 2D truss system where each truss has only one element

    create TrussElement2D(node_indeces, E,A,L, f=[0,0,0,0], theta = 0)
        - k_elem 4x4


"""

'\nTODOs:\n    solve for displacements of a 2D truss system where each truss has only one element\n    \n    create TrussElement2D(node_indeces, E,A,L, f=[0,0,0,0], theta = 0)\n        - k_elem 4x4\n\n\n'

In [31]:
# class Node:
#     def __init__(self, index):
#         self.index = index
#         self.coordinates = None

# # TODO: Should I use node coordinates for a node or can i get

In [35]:
class TrussElement2D:
    def  __init__(self, node_indeces, node_coordinates, E,A,L, distributed_load = 0, f_applied=[0,0,0,0], theta = 0, shape_function = "linear"):
        self.node_indeces = node_indeces
        self.node_coordinates = node_coordinates
        self.E = E
        self.A = A
        self.L = L
        self.q = distributed_load
        self.f_applied = f_applied
        self.theta = theta # from (right) to (up)
        self.shape_function = shape_function

        self.get_local_stiffness_matrix()
        self.get_global_stiffness_matrix()

    def get_local_stiffness_matrix(self):
        """
        returns the stiffness matrix for the element in local coordinates
        """

        if self.shape_function == "linear":
            k  = self.E*self.A/self.L
            self.k_elem_local = k * np.array([[1,0,-1,0],[0,0,0,0],[-1,0,1,0],[0,0,0,0]])

    def get_global_stiffness_matrix(self):
        """
        returns the global stiffness matrix for the element in global coordinates
        """
        S = np.sin(np.deg2rad(self.theta))
        C = np.cos(np.deg2rad(self.theta))

        R = np.array([[C, S],
                      [-S, C]])

        T = np.array([[C,S,0,0],
                      [-S,C,0,0],
                      [0,0,C,S],
                      [0,0,-S,C]])

        self.K_elem_global = T.T @ self.k_elem_local @ T
        return self.K_elem_global




In [33]:
class TrussSystem2D:
    def __init__(self, elements):
        self.elements = elements

    def get_node_coordinates(self):
        origin = [0,0]

        for element in self.elements:
            for node_index in element.node_indeces:
                if node_index == 0:
                    node_coordinates = origin



In [25]:
class Plotter:
    def __init__(self, system):
        self.system = system

    def plot_system(self):
        """
        plots the truss system
        """
        for element in self.system.elements:
            node1, node2 = element.node_indeces



            plt.plot([x1,x2], [y1,y2])
        plt.show()


In [ ]:
E = 100*10^3; A = 625; L=2000; # for each truss element, [units: N, mm, MPa]

connectivity = np.array([[0,1], [0,2], [1,2], [1,3], [2,3], [2,4], [3,4]])
nodal_coordinates = np.array([[0,0], [0,L], [L,0], [L,L], [2*L,0]])
boundary_conditions = {0:np.array([1,0]),  # key = node_index, value = constraints in xy
                       1:np.array([1,1])}

elements = []
for i, node_pair in enumerate(connectivity):
    node1, node2 = node_pair
    element = TrussElement2D([node1, node2], E,A,L, f_applied=[0,0,0,0], theta = 0)
    elements.append(element)

system = TrussSystem2D(elements)

# Define elements with consistent properties
elements = [
    TrussElement2D([nodes[0], nodes[1]], E, A, L, f_applied=[0, 0, 0, 0], theta=90),
    TrussElement2D([nodes[0], nodes[2]], E, A, L, f_applied=[0, 0, 0, 0], theta=0),
    TrussElement2D([nodes[1], nodes[2]], E, A, L, f_applied=[0, 0, 0, 0], theta=135),
    TrussElement2D([nodes[1], nodes[3]], E, A, L, f_applied=[0, 0, 0, 0], theta=0),
    TrussElement2D([nodes[2], nodes[3]], E, A, L, f_applied=[0, 0, 0, 0], theta=90),
    TrussElement2D([nodes[2], nodes[4]], E, A, L, f_applied=[0, 0, 0, 0], theta=0),
    TrussElement2D([nodes[3], nodes[4]], E, A, L, f_applied=[0, 0, 0, 0], theta=135)
]



In [34]:
E = 100*10^3; A = 625; L=2000; # for each truss element, [units: N, mm, MPa]

element1 = TrussElement2D([0,1], E,A,L, f_applied=[0,0,0,0], theta = 90)
element2 = TrussElement2D([0,2], E,A,L, f_applied=[0,0,0,0], theta = 0)
element3 = TrussElement2D([1,2], E,A,L, f_applied=[0,0,0,0], theta = 135)
element4 = TrussElement2D([1,3], E,A,L, f_applied=[0,0,0,0], theta = 0)
element5 = TrussElement2D([2,3], E,A,L, f_applied=[0,0,0,0], theta = 90)
element6 = TrussElement2D([2,4], E,A,L, f_applied=[0,0,0,0], theta = 0)
element7 = TrussElement2D([3,4], E,A,L, f_applied=[0,0,0,0], theta = 135)



elements = [element1, element2, element3, element4, element5, element6, element7]

system = TrussSystem2D(elements)
plotter = Plotter(system)


In [27]:
plotter = Plotter(system)
plotter.plot_system()

NameError: name 'x1' is not defined

In [15]:
for element in system.elements:
    pprint.pp(element.K_elem_global)

array([[ 1.17520239e-30,  1.91925116e-14, -1.17520239e-30,
        -1.91925116e-14],
       [ 1.91925116e-14,  3.13437500e+02, -1.91925116e-14,
        -3.13437500e+02],
       [-1.17520239e-30, -1.91925116e-14,  1.17520239e-30,
         1.91925116e-14],
       [-1.91925116e-14, -3.13437500e+02,  1.91925116e-14,
         3.13437500e+02]])
array([[ 313.4375,    0.    , -313.4375,    0.    ],
       [   0.    ,    0.    ,    0.    ,    0.    ],
       [-313.4375,    0.    ,  313.4375,    0.    ],
       [   0.    ,    0.    ,    0.    ,    0.    ]])
array([[ 156.71875, -156.71875, -156.71875,  156.71875],
       [-156.71875,  156.71875,  156.71875, -156.71875],
       [-156.71875,  156.71875,  156.71875, -156.71875],
       [ 156.71875, -156.71875, -156.71875,  156.71875]])
array([[ 313.4375,    0.    , -313.4375,    0.    ],
       [   0.    ,    0.    ,    0.    ,    0.    ],
       [-313.4375,    0.    ,  313.4375,    0.    ],
       [   0.    ,    0.    ,    0.    ,    0.    ]])
arra

In [ ]:
element1.nodes